In [1]:
import pickle 
import sys 
import os 
import matplotlib.pyplot as plt 
# sys.path.append(os.path.abspath('../../../lammps-21Nov2023/multiscale_project/checkpoints/'))
from glob import glob
sys.path.append(os.path.abspath('../../checkpoints'))
sys.path.append(os.path.abspath('../../src'))
from ea import EA
from ea import Material 
from ea import Grain
import numpy as np

In [20]:
seeds = range(0,3)
period_pairs = [[0.0666,0.05]]
fig, axes = plt.subplots(1, len(seeds), sharey=True, sharex=True, figsize=(20, 10))
eval_method = 'simult'
gate_1 = "XOR"
gate_2 = "AND"

# checkpoints/seq_eval_run0_period0.0666_period0.05_1gens_AND_AND.p
for pindx, period_pair in enumerate(period_pairs):
    for seed_index, seed in enumerate(seeds):
        filenames = glob(f'../final_checkpoints/{eval_method}_eval_run{seed}_period{period_pair[0]}_period{period_pair[1]}_*gens_{gate_1}_{gate_2}.p')
        filenames.sort(key=lambda x: int(x.split(f"period{period_pair[1]}_")[1].split("gens")[0]))
        
        filenames=filenames[:50]
        data = np.zeros((len(filenames), 100, 2))

        for file_index, filename in enumerate(filenames):
            with open(filename, 'rb') as f:
                # Load the data from the file
                ea, rng_state, np_rng_state = pickle.load(f)
            xs = []
            ys = []
            for org_index, org in enumerate(ea.population):
                #print(org.fitness1, org.fitness2)
                data[file_index,org_index,0] = org.fitness1
                data[file_index,org_index,1] = org.fitness2
        colors = plt.cm.viridis(np.linspace(0, 1, len(filenames)))
        for i in range(len(filenames)):
            s= axes[seed_index].scatter(data[i, :, 0], data[i, :, 1], color=colors[i])

        axes[seed_index].set_title(f"Gens {len(filenames)*2} Seed {seed} periods {period_pair[0]}, {period_pair[1]}")
        axes[seed_index].axvline(x=1, color='red', linestyle='--')
        axes[seed_index].axhline(y=1, color='red', linestyle='--')
        axes[seed_index].set_aspect('equal', adjustable='box')
        axes[seed_index].set_xlim([0, 10])
        axes[seed_index].set_ylim([0, 10])



plt.xlabel(f"{gate_1}ness f1")
plt.ylabel(f"{gate_2}ness f2")
plt.tight_layout()

plt.show()
